In [2]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
bc = datasets.load_breast_cancer()
x,y = bc.data, bc.target
n_samples, n_features = x.shape

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=1234)

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test  = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input, 1)

    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [5]:
model = LogisticRegression(n_features)

In [6]:
loss = nn.BCELoss()

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=.01)

In [8]:
for epoch in range(100):
    y_predicted = model(x_train)

    l = loss(y_predicted, y_train)

    l.backward()

    optimizer.step()

    optimizer.zero_grad()

    if (epoch+1)%10 == 0:
        print(f"epoch:{epoch+1}, loss:{l.item():.3f}")

epoch:10, loss:0.529
epoch:20, loss:0.435
epoch:30, loss:0.376
epoch:40, loss:0.335
epoch:50, loss:0.306
epoch:60, loss:0.283
epoch:70, loss:0.265
epoch:80, loss:0.250
epoch:90, loss:0.237
epoch:100, loss:0.227


In [9]:
with torch.no_grad():
    y_predicted = model(x_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(acc)

tensor(0.9298)
